In [1]:
import csv
import duckdb

In [2]:
op = open("/home/factentry/otto_ml/src/antlr/mondal/pdf_table_extractor/emma_jul29_op.csv", "w+", encoding="utf8")

In [3]:
fop = open("/home/factentry/otto_ml/src/antlr/mondal/pdf_table_extractor/emma_jul29_skipped.csv", "w+", encoding="utf8")

In [4]:
con = duckdb.connect()

In [5]:
con.sql("CREATE TABLE emma_file_status AS FROM '/home/factentry/otto_ml/src/antlr/mondal/pdf_table_extractor/emma_file_status.csv';") 

In [6]:
res = con.sql("select * from emma_file_status")
res

┌───────────────────────────────┬────────────────────┬───────────┐
│           FileName            │      Comment       │ PageCount │
│            varchar            │      varchar       │   int64   │
├───────────────────────────────┼────────────────────┼───────────┤
│ MS253781-MS229089-MD446469    │ File doesn't exist │      NULL │
│ ﻿EP681764-EP531149-EP932386    │ File doesn't exist │      NULL │
│ MS260348-MS235656-MD459704    │ File doesn't exist │      NULL │
│ EP371655-EP292093-EP687552    │ File doesn't exist │      NULL │
│ MS276953-MS274991-MD557919    │ File doesn't exist │      NULL │
│ MS133665-MS108973-MD210890    │ File doesn't exist │      NULL │
│ ER823649-ER641859-ER1043756   │ File doesn't exist │      NULL │
│ MS172331-MS147639-MD284873    │ File doesn't exist │      NULL │
│ MS64259-MS266360-MD521900     │ File doesn't exist │      NULL │
│ MS124813-MS100121-MD524476    │ File doesn't exist │      NULL │
│             ·                 │  ·                 │       

In [7]:
len(res)

167881

In [8]:
writer = csv.DictWriter(op, fieldnames=['FileName','cusip','maturitydate','amountissued','interest_rate','issue_price','yield','redemption_date', 'Comment'])
writer.writeheader()

98

In [11]:
import sys
sys.path.insert(0,'/home/factentry/otto_ml/src')
from utils import db_class, base_class

from tqdm import tqdm

# FERACK =  db_class.Database_Manager('FERACK','FE1')
FERACK13 =  db_class.Database_Manager('FERACK13','SPIDER')

***** FERACK13 SPIDER Database connection established *****


In [12]:
with open("/home/factentry/otto_ml/src/antlr/mondal/pdf_table_extractor/FileLink_Jul29.csv", encoding="utf8") as f:
    for fileid in tqdm(f):
        fileid = fileid.strip().upper()
        status = con.sql(f"select * from emma_file_status where FileName = '{fileid}' and PageCount is NULL")
        if len(status):
            writer.writerow({
                'FileName': fileid,
                'Comment': 'Failed to open file'
            })
            continue

        query = f"""select * from spider.dbo.emma_table_extraction where filename='{fileid}'"""
        isin_queue = FERACK13.fetchArray_withKey(query)

        if not isin_queue:
            print(fileid, file=fop)
        for row in isin_queue:
            writer.writerow(row)


3109it [01:03, 48.63it/s]


In [ ]:
op.close()

In [ ]:
fop.close()

In [ ]:
con.close()

In [1]:
import duckdb
import pandas as pd
from table_extractor import table_extractor
import muni_table_standardiser as standardiser
from muni_utils import table_inserter

In [2]:
id = "EA288849-EA4196-EA626568"
PDF_FILE = f"/FERack11_FE_documents2/EMMA_Official_Statement/{id}.pdf"

extractor = table_extractor.TableExtractor(PDF_FILE)

In [3]:
df_map = extractor.extract_tables(page_range=[1])

2.8764044943820224
3.3529411764705883
3.3529411764705883
81.33333333333333
Skipping small bbox
82.66666666666667
Skipping small bbox
82.0
Skipping small bbox
82.0
Skipping small bbox
80.66666666666667
Skipping small bbox
82.66666666666667
Skipping small bbox
2.842696629213483
3.2941176470588234
3.287581699346405
Area: (88, 54, 266, 566) of page no 2
[       due Unnamed: 1 principaL interest Unnamed: 4 Unnamed: 5   due.1  \
0   oct. 1        NaN    aMount     rate      yieLd      cusip  oct. 1   
1     2010          $   630,000    3.00%      0.71%  914720TK3    2021   
2     2011  1,355,000       NaN     3.00       1.44  914720TL1    2022   
3     2012  1,395,000       NaN     4.00       1.85  914720TM9    2023   
4     2013        NaN   620,000     4.50       2.34  914720TN7    2023   
5     2013        NaN   840,000     5.00       2.34  914720TP2    2024   
6     2014  1,515,000       NaN     3.00       2.72  914720TQ0    2024   
7     2015  1,580,000       NaN     4.00       2.96  91

In [6]:
type(df_map)

dict

In [9]:
df_map

{1: [       due Unnamed: 1 principaL interest Unnamed: 4 Unnamed: 5   due.1  \
  0   oct. 1        NaN    aMount     rate      yieLd      cusip  oct. 1   
  1     2010          $   630,000    3.00%      0.71%  914720TK3    2021   
  2     2011  1,355,000       NaN     3.00       1.44  914720TL1    2022   
  3     2012  1,395,000       NaN     4.00       1.85  914720TM9    2023   
  4     2013        NaN   620,000     4.50       2.34  914720TN7    2023   
  5     2013        NaN   840,000     5.00       2.34  914720TP2    2024   
  6     2014  1,515,000       NaN     3.00       2.72  914720TQ0    2024   
  7     2015  1,580,000       NaN     4.00       2.96  914720TR8    2025   
  8     2016  1,650,000       NaN     5.00       3.20  914720TS6    2026   
  9     2017  1,730,000       NaN     3.75       3.46  914720TT4    2026   
  10    2018  1,800,000       NaN     4.25       3.66  914720TU1    2027   
  11    2019  1,495,000       NaN     5.00       3.82  914720TV9    2028   
  12    2